In [5]:
import requests
import time
import json

from inspirehep import *

In [6]:
f=open('db_LA.json','r')
db=json.load(f)
f.close()
idsLA=[d.get('LA_institution_id') for d in db]
idsAU=[d.get('author_id') for d in db]
db_index=get_index(db)

In [7]:
db_inst_index=get_index(db,index='LA_institution_id')

In [8]:
collaborations=[]
collaborations_old=[]

Pagination

In [9]:
page=0
if len(collaborations)>0 and  collaborations_old!=collaborations:
    cn_search='https://inspirehep.net/api/literature?sort=mostrecent&size=25&page=1&q=ac>100+and+de>2018'
    cn=f"{cn_search}+and+not+cn+{'+and+not+cn+'.join(collaborations)}"
else:
    #increase pagination
    if page==0:
        extra=''
    else:
        extra='+and+not+cn+'
    page=page+1
    cn_search=f'https://inspirehep.net/api/literature?sort=mostrecent&size=25&page={page}&q=ac>100+and+de>2018'
    cn=f"{cn_search}{extra}{'+and+not+cn+'.join(collaborations)}"

In [10]:
r=requests.get(cn)

In [11]:
d=r.json()

In [12]:
d.get('hits').get('total')

2520

In [13]:
l=d.get('hits').get('hits')

Not affilation in `'authors'` list for collaborations!

In [14]:
l=l[1:]

In [15]:
for ld in l:
    lit=ld.get('metadata')
    if not lit.get('collaborations'):
        continue # next paper
    try:
        collaboration=lit.get('collaborations')[0].get('value')
        FoS=[collaboration]
    except:
        collaboration=''
        FoS=[]

    if collaboration in collaborations:
        continue #next paper
        
    try:
        PAC=lit['primary_arxiv_category']
    except:
        PAC=[]
    la=lit.get('authors')
    for a in la:   
        CONTINUE,author_id,url_author,FoS,PAC=get_work(db,a,idsLA,idsAU,db_index,FoS,PAC)
        if CONTINUE:
            continue #next author
        positions,email_addresses,name,aids=get_author(url_author) #USES API!
        try:
            inst_id=[d.get('record').get('$ref').split('/')[-1] 
                 for d in positions if d.get('record') and d.get('current')][0]
        except:
            inst_id=None
        if inst_id not in idsLA:
            continue #next author
        di=db[db_inst_index[inst_id]]
        
        new={'author_id':author_id,
                 'name':name,
                 'LA_institution_id':inst_id,
                 'external_system_identifiers':di.get('external_system_identifiers'),
                 'LA_country':di.get('LA_country'),
                 'LA_institution':di.get('LA_institution'),
                 'email_addresses':email_addresses,
                 'positions':positions,
                 'ids':aids,
                 'primary_arxiv_category':PAC,
                 'Fields of Study': FoS # in ['Particle physics','Cosmology','Astrophysics','Mathematical physics']
                }
        db.append(new)
        break
    collaborations.append(collaboration) #Only after search for all the authors        
    break

NameError: name 'time' is not defined

In [1]:
import json

In [2]:
f=open('data/db_LA.json','r')
df=json.load(f)
f.close()

In [4]:
f=open('db_LA.json','r')
dfn=json.load(f)
f.close()

In [14]:
df[''primary_arxiv_category']

{'author_id': '1876618',
 'name': {'value': 'Ramírez, Valeria'},
 'LA_institution_id': '909015',
 'external_system_identifiers': [{'value': 'grid.412866.f', 'schema': 'GRID'},
  {'value': 'https://ror.org/031f8kt38', 'schema': 'ROR'},
  {'value': 'INST-77020', 'schema': 'SPIRES'}],
 'LA_country': 'Mexico',
 'LA_institution': 'UAEH, Pachuca',
 'email_addresses': [],
 'positions': [],
 'ids': [{'value': 'V.Ramirez.1', 'schema': 'INSPIRE BAI'}],
 'primary_arxiv_category': ['gr-qc'],
 'Fields of Study': ['Mathematical physics']}

In [21]:
[d.get('primary_arxiv_category') for d in df][30:40]

[['nlin.AO',
  'cond-mat.str-el',
  'cond-mat.other',
  'hep-ex',
  'hep-ph',
  'astro-ph.GA',
  'physics.gen-ph',
  'hep-th',
  'astro-ph.HE',
  'hep-lat',
  'quant-ph',
  'math-ph',
  'gr-qc',
  'cond-mat.stat-mech',
  'astro-ph.CO',
  'astro-ph.SR',
  'nucl-th',
  'physics.pop-ph'],
 ['cond-mat.stat-mech',
  'cond-mat.str-el',
  'hep-ex',
  'hep-ph',
  'hep-th',
  'quant-ph',
  'gr-qc',
  'math-ph'],
 ['math-ph', 'hep-th'],
 ['math-ph', 'hep-th'],
 ['hep-ph'],
 ['hep-ph'],
 ['hep-ph'],
 ['hep-ph', 'astro-ph.CO'],
 ['hep-ph'],
 ['hep-th']]

In [20]:
[d.get('primary_arxiv_category') for d in dfn][30:40]

[['hep-th', 'hep-ph', 'gr-qc'],
 ['hep-th', 'hep-ph', 'gr-qc'],
 ['hep-th'],
 ['hep-th'],
 ['hep-ph'],
 ['hep-ph'],
 ['hep-ph'],
 ['astro-ph.CO', 'hep-ph'],
 ['hep-ph'],
 ['hep-th']]